In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os

In [ ]:
df = pd.read_excel("/content/data.xlsx")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df = df.sort_values(by="Date")
df

In [ ]:
df.index = df.Date

In [ ]:
plt.figure(figsize=(10,3))
df.XXX.plot()

In [ ]:
plt.figure(figsize=(10,3))
df.Tehran.plot()

In [ ]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
temp = df.XXX

In [ ]:
WINDOW_SIZE = 7
X1, y1 = df_to_X_y(temp, WINDOW_SIZE)
X1.shape, y1.shape

In [ ]:
X_train1, y_train1 = X1[:750], y1[:750]
X_val1, y_val1 = X1[750:800], y1[750:800]
X_test1, y_test1 = X1[800:], y1[800:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
seed = 7
tf.random.set_seed(seed)
model1 = Sequential()
model1.add(InputLayer((WINDOW_SIZE, 1)))
model1.add(LSTM(64))
#model1.add(LSTM(64))
#model1.add(Dense(8, 'tanh'))
#model1.add(Dense(1, 'linear'))
model1.add(Dense(1))
model1.summary()

In [ ]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])

In [ ]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50, callbacks=[cp1])

In [ ]:
# from tensorflow.keras.models import load_model
# model1 = load_model('model1/')

In [ ]:
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

In [ ]:

train_results["Actuals"].plot()
train_results["Train Predictions"].plot()

In [ ]:
val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})
val_results

In [ ]:
plt.plot(val_results['Val Predictions'],label="Predict")
plt.plot(val_results['Actuals'],label="actual")
plt.legend()


In [ ]:
test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
test_results

In [ ]:
plt.figure(figsize=(11,5))
plt.plot(test_results['Test Predictions'],label="Prediction")
plt.plot(test_results['Actuals'],label="actual")

#one_layer_LSTM=test_results['Test Predictions']
#plt.plot(one_layer_LSTM.shift(-23),label="past")
plt.legend()

In [ ]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
in_seq1 = array(df.XXX)
in_seq2 = array(df.YYY)
#out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
y

In [ ]:
X_train1, y_train1 = X[:750], y[:750]
X_val1, y_val1 = X[750:800], y[750:800]
X_test1, y_test1 = X[800:], y[800:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

In [ ]:
seed = 7
tf.random.set_seed(seed)
model1 = Sequential()
model1.add(InputLayer((n_steps, n_features)))
model1.add(LSTM(64))
#model1.add(LSTM(64))
#model1.add(Dense(8, 'tanh'))
#model1.add(Dense(1, 'linear'))
model1.add(Dense(1))
model1.summary()

In [ ]:
model1.compile(optimizer='adam', loss='mse')
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50, callbacks=[cp1])


In [ ]:
test_predictions = model.predict(X_test1)
#test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
#test_results

In [ ]:
len(test_predictions)

In [ ]:
y_test1[0]

In [ ]:
test_predictions[0]

In [ ]:
plt.figure(figsize=(11,5))
plt.plot(test_predictions,label="Prediction")
plt.plot(y_test1,label="actual")


plt.legend()